In [1]:
# TODO which one?
#git clone https://github.com/lucidrains/iTransformer.git
#import iTransformer
import sys
sys.path.append('/vol/fob-vol7/nebenf21/reinbene/bene/MA/iTransformer') 
from iTransformer import iTransformer

import torch
import torch.optim as optim
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import pandas as pd
from pathlib import Path

window_size = 96
pred_length = (96)

from utils import data_handling, training_functions, helpers
import config 

print("Import succesfull")

# use full train dataset for training or small 4% subset
four_weeks = -24*7*4

Import succesfull


# Benchmark all datasets used for transfer learning on iTransformer on its own.

After the sanity check, we will use 96h as the input and the prediction horizon. In the following we will use the RevIn normalization strategie to benchmark iTransformer on our transfer learning datasets to get an initial baseline. RevIn is used because if performed the best on our initial baseline. This baseline will be compared to an SARIMA implementation and an additional DL model.

Those results will also be used to evaluate the transfer-learning capability of iTransformer.

Because transfer-learning is a sensible solution for the cold-start problem, we also do benchmarks on iTransformer only trained on the first 10% of the train dataset. Beacuse all datasets are big enough for efficient training, using a small subset is a solutoin to get meaningfull insight in how much value can be created through transfer-learning. 

In [2]:
# use electricity dataset
data_dict = data_handling.load_electricity()

electricity = {}
electricity["dataloader_train"], electricity["dataloader_validation"], electricity["dataloader_test"] = data_handling.convert_data(data_dict, window_size, pred_length)
data_dict["train"].shape

# create a smaller subset of the train dataset
electricity["4_weeks_train"] = data_dict["train"][four_weeks:,:]
electricity["4_weeks_train"] = data_handling.SlidingWindowTimeSeriesDataset(electricity["4_weeks_train"] , window_size, pred_length)
electricity["4_weeks_train"] = data_handling.DataLoader(electricity["4_weeks_train"] , batch_size=32, shuffle=True)

Feature batch shape: torch.Size([32, 96, 348])


In [2]:
# building genome project dataset
data_tensor = data_handling.load_genome_project_data()
gp_dict, standadizer = data_handling.train_test_split_eu_elec(data_tensor, standardize=True)

# convert to dataloader
genome_project = {}
genome_project["dataloader_train"], genome_project["dataloader_validation"], genome_project["dataloader_test"] = data_handling.convert_data(gp_dict, window_size, pred_length)

# create a smaller subset of the train dataset
genome_project["4_weeks_train"] = gp_dict["train"][four_weeks:,:]
genome_project["4_weeks_train"] = data_handling.SlidingWindowTimeSeriesDataset(genome_project["4_weeks_train"] , window_size, pred_length)
genome_project["4_weeks_train"] = data_handling.DataLoader(genome_project["4_weeks_train"] , batch_size=32, shuffle=True)


Feature batch shape: torch.Size([32, 96, 1454])


In [4]:
# bavaria dataset
data_tensor = data_handling.load_bavaria_electricity()
data_dict, standadizer = data_handling.train_test_split_eu_elec(data_tensor, standardize=True)

# convert to datalaoder
bavaria = {}
bavaria["dataloader_train"], bavaria["dataloader_validation"], bavaria["dataloader_test"] = data_handling.convert_data(data_dict, window_size, pred_length)

# create a smaller subset of the train dataset
bavaria["4_weeks_train"] = data_dict["train"][four_weeks:,:]
bavaria["4_weeks_train"] = data_handling.SlidingWindowTimeSeriesDataset(bavaria["4_weeks_train"] , window_size, pred_length)
bavaria["4_weeks_train"] = data_handling.DataLoader(bavaria["4_weeks_train"] , batch_size=32, shuffle=True)

Feature batch shape: torch.Size([32, 96, 59])


In [3]:
# run experiment for each dataset and save model and evaluation metrics
dataset_dict = {
              #  "electricity": electricity,
             #   "bavaria": bavaria,
                "genome_project" : genome_project,                
                }


def train_and_evaluate(dataset_dict, dataset_name, full_dataset= True, epoch=20):

    if full_dataset == False:
        print("Selecting 4 week dataset")
        training_dataloader = dataset_dict["4_weeks_train"]
    else:
        training_dataloader = dataset_dict["dataloader_train"]
        
    inputs, _ = next(iter(training_dataloader))
    num_variates = inputs.size(2)
    
    # define parameters and create config 
    best_parameters = {'depth': 2, 'dim': 256, 'dim_head': 56, 'heads': 4, 'attn_dropout': 0.2, 'ff_mult': 4, 'ff_dropout': 0.2, 
                    'num_mem_tokens': 4, 'learning_rate': 0.0005}

    best_parameters = {'depth': 2, 'dim': 512, 'dim_head': 56, 'heads': 4, 'attn_dropout': 0.2, 'ff_mult': 4, 'ff_dropout': 0.2, 
                    'num_mem_tokens': 4, 'learning_rate': 0.0005}

    model_config = {
        'num_variates': num_variates,
        'lookback_len': window_size,
        'depth': best_parameters["depth"],
        'dim': best_parameters["dim"],
        'num_tokens_per_variate': 1,
        'pred_length': pred_length,
        'dim_head': best_parameters["dim_head"],
        'heads': best_parameters["heads"],
        'attn_dropout': best_parameters["attn_dropout"],
        'ff_mult': best_parameters["ff_mult"],
        'ff_dropout': best_parameters["ff_dropout"],
        'num_mem_tokens': best_parameters["num_mem_tokens"],
        'use_reversible_instance_norm': True,
        'reversible_instance_norm_affine': True,
        'flash_attn': True
    }

    # select available deviec
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    print(f"Using device: {device}")

    # defining all needed instances
    model = iTransformer(**model_config).to(device)
    optimizer = optim.Adam(model.parameters(), lr=best_parameters["learning_rate"])
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    writer = SummaryWriter(log_dir=config.CONFIG_LOGS_PATH["iTransformer_baseline"] / dataset_name)

    # run model training as mentioned in the original paper
    if full_dataset == False:
        checkpoint_path = config.CONFIG_MODEL_LOCATION["iTransformer_baseline"] / dataset_name / f"{dataset_name}_4_weeks_best_val_loss.pt"
    else:
        checkpoint_path = config.CONFIG_MODEL_LOCATION["iTransformer_baseline"] / dataset_name / f"{dataset_name}_full_dataset_best_val_loss.pt"


    # load model with best validaiton mse
    try:
        checkpoint = torch.load(checkpoint_path)
        model = iTransformer(**model_config).to(device)
        model.load_state_dict(checkpoint['model_state_dict'])
        epoch = epoch - checkpoint["epoch"]
        if epoch == 0:
            print("Model is already trained for 20 epochs.")
            return None
    except:
        print("Training from scratch.")

    train_metrics, best_model = training_functions.train_one_epoch(epoch, model, device, training_dataloader, dataset_dict["dataloader_validation"], \
                                            optimizer, scheduler, writer, checkpoint_path)


    # predict on test set
    metrics = helpers.full_eval(best_model, dataset_dict["dataloader_test"], device)
    for eval_metric, value in metrics[96].items():
        metrics[96][eval_metric] = value.item()


    metrics_df = pd.DataFrame.from_dict(metrics[96], orient='index')
    metrics_df.rename(columns={0: dataset_name}, inplace=True)

    if full_dataset == False:
        metrics_df.to_csv(f"{config.CONFIG_OUTPUT_PATH['iTransformer_baseline']}/metrics_{dataset_name}_epochs{epoch}_4_week_dataset.csv")
    else:
        metrics_df.to_csv(f"{config.CONFIG_OUTPUT_PATH['iTransformer_baseline']}/metrics_{dataset_name}_epochs_{epoch}baseline.csv")


In [7]:
#for key, value in dataset_dict.items():
#    train_and_evaluate(value, key, full_dataset=False, epoch=20)

In [4]:
for key, value in dataset_dict.items():
    print(key)
    train_and_evaluate(value, key, full_dataset=True, epoch=10)

genome_project
Using device: cuda
Non-A100 GPU detected, using math or mem efficient attention if input tensor is on cuda


Epoch: Validating: 100%|██████████| 40/40 [00:10<00:00,  3.85it/s]


Validation metrics: {'mse': tensor(1.5894, device='cuda:0')}
Checkpointing succesfull after epoch 1


Epoch: Validating: 100%|██████████| 40/40 [00:43<00:00,  1.08s/it]


Validation metrics: {'mse': tensor(1.5894, device='cuda:0')}


...

Epoch: Validating: 100%|██████████| 40/40 [00:12<00:00,  3.12it/s]


Validation metrics: {'mse': tensor(1.5894, device='cuda:0')}


Epoch: Validating: 100%|██████████| 40/40 [00:12<00:00,  3.32it/s]


Validation metrics: {'mse': tensor(1.5894, device='cuda:0')}


Epoch: Validating: 100%|██████████| 86/86 [00:22<00:00,  3.88it/s]
